## Etapa 7: Treinando o modelo com uma RNN

Por fim, faremos uma nova predição do mesmo, porém utilizando uma rede neural recorrente, normalmente mais indicadas para trabalhar com texto.

In [1]:
# importa funções padroes
import pandas as pd 
import numpy as np
import re

In [2]:
# importa funções para redes neurais
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [3]:
def predicao(lista_texto):
    texto_token = tokenizer.texts_to_sequences(lista_texto)
    texto_padded_tokens = sequence.pad_sequences(texto_token, maxlen=100)
    return (model.predict(texto_padded_tokens[:len(lista_texto)]) > .5).astype(int)
    #return model.predict(texto_padded_tokens[:len(lista_texto)]).argmax(1)

def predicao_pesos(lista_texto):
    texto_token = tokenizer.texts_to_sequences(lista_texto)
    texto_padded_tokens = sequence.pad_sequences(texto_token, maxlen=100)
    return model.predict(texto_padded_tokens[:len(lista_texto)])

In [4]:
#importa dataset
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [14]:
X_train = train['content'].values.astype(str)
X_test = test['content'].values.astype(str)
y_train = train['score']
y_test = test['score']

In [6]:
max_features = 40000
maxlen = 250

In [15]:
# treina um tokenizador nos dados de treino
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
len(tokenizer.word_index)

44194

In [16]:
# tokeniza os dados
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [17]:
# corta ou adiciona zeros em sequências maiores de 250
X_train_padded = sequence.pad_sequences(X_train_token, maxlen=maxlen)
X_test_padded = sequence.pad_sequences(X_test_token, maxlen=maxlen)
X_train_padded.shape, X_test_padded.shape

((165869, 250), (71088, 250))

In [18]:
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [19]:
# monta o modelo de rede neural
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 64)           2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, 250, 64)           24832     
_________________________________________________________________
dropout (Dropout)            (None, 250, 64)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,609,729
Trainable params: 2,609,729
Non-trainable params: 0
______________________________________________

In [26]:
# treina a rede neural
model.fit(x=X_train_padded, y=y_train, batch_size=256, epochs=7)

Epoch 1/5
648/648 [==============================] - 1994s 3s/step - loss: 0.1053 - accuracy: 0.9650
Epoch 2/5
648/648 [==============================] - 1977s 3s/step - loss: 0.0917 - accuracy: 0.9701
Epoch 3/5
648/648 [==============================] - 1968s 3s/step - loss: 0.0816 - accuracy: 0.9738
Epoch 4/5
648/648 [==============================] - 1950s 3s/step - loss: 0.0736 - accuracy: 0.9763
Epoch 5/5
648/648 [==============================] - 1974s 3s/step - loss: 0.0656 - accuracy: 0.9787


In [27]:
#comparar os resultados
acc = model.evaluate(x=X_test_padded, y=y_test)

2222/2222 [==============================] - 129s 58ms/step - loss: 0.1796 - accuracy: 0.9486


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Modelo salvo em disco!")

Rotulando o dataset dos Tweets com a previsão do modelo RNN

In [ ]:
# importando dados dos tweets
dataset = pd.read_csv('dataset_com_sentimento.csv', index_col=0)

In [ ]:
# realizando a predição na base dos tweets
y_pred = predicao(dataset.full_text)
y_pred_score = predicao_pesos(dataset.full_text)

In [ ]:
dataset['sentimento_RNN'] = y_pred
dataset['pesos_RNN'] = y_pred_score

In [ ]:
dataset.head()

In [ ]:
dataset.to_csv('dataset_com_sentimento.csv')

print('Arquivo Gravado com Sucesso')